In [1]:
import datetime
import re

from bs4 import BeautifulSoup
import requests

In [2]:
def get_category_urls(category_url):
    i = 0; result = []
    while True:
        i += 1
        page_url = category_url+'&p='+str(i)
        soup = BeautifulSoup(requests.get(category_url).text,'html.parser')
        urls = ['http://www.setn.com'+soup.select("ol a")[i].attrs['href'] for i in range(len(soup.select("ol a")))]
        result.extend(urls)
        if len(urls)==0 or i>30:
            break
    
    return result

In [3]:
url = 'http://www.setn.com/ViewAll.aspx?pagegroupid=5'
urls = get_category_urls(url)
len(urls)

930

In [4]:
def parser_page(url):
    """
    parser page method 
    主要用於抓取新聞detail頁面資訊

    page_data = parser_page(news detail url)
    """
    soup = BeautifulSoup(requests.get(url).text,'html.parser')
    search = lambda typ,seed: soup.find_all("meta", attrs={typ : seed})[0].attrs['content']
    inputs = [("property","og:url"),("property","og:title"),("name","pubdate"),("name","Keywords"),("name","section")]
    
    parse = [search(typ,seed) for typ,seed in inputs]
    post_time = datetime.datetime(*map(lambda x: int(x),re.findall('[0-9]+',parse[2])))
    
    article = soup.select("article div > p")
    
    content = []; journalist = None
    for i in range(len(article)):
        if len(article[i].attrs) == 0 and not journalist:
            journalist = article[i].text
        else:
            content.append(article[i].text)   
    content = "".join(content)
    
    fb_url = 'https://graph.facebook.com/fql?q=SELECT%20like_count,%20total_count,%20share_count, \
              %20click_count,%20comment_count%20FROM%20link_stat%20WHERE%20url%20=%20%22'+parse[0]+'%22'
    
    res_data = requests.get(fb_url).json()['data'][0]
    fb_like = res_data['like_count']
    fb_share = res_data['share_count']
    fb_com = res_data['comment_count']
    
    ######facebook comments#######   
    tree = {}
    fb_com_url = 'http://graph.facebook.com/comments?id=http://www.setn.com/News.aspx?NewsID='+ \
                  re.findall('NewsID=([0-9]+)',parse[0])[0]+ \
                 '&limit=100&filter=stream&fields=parent.fields%28id%29,message,from,like_count,created_time,parent'
    fb_data = requests.get(fb_com_url).json()
    
    while True:
        if len(fb_data["data"])==0:
            break
        for comment in fb_data['data']:
            one_com = {}
            one_com["post_time"] = datetime.datetime(*map(lambda x: int(x),re.findall('[0-9]+',comment['created_time'][:-5])))
            one_com["actor"] = comment['from']['name']
            one_com["like"] = comment['like_count']
            one_com["content"] = comment['message']
            one_com["source_type"] = "facebook"
            one_com["dislike"] = None
            if 'parent' in comment:
                tree[comment['parent']['id']]["sub_comments"].append(one_com)
            else:
                one_com["sub_comments"] = []
                tree[comment['id']] = one_com

        fb_next_url = fb_com_url+'&after='+fb_data['paging']['cursors']['after']
        fb_data = requests.get(fb_next_url).json()
    ######facebook comments#######        
    
    
    return {
        "url": parse[0],
        "source_press": None,
        "title": parse[1],
        "post_time": post_time,
        "journalist": journalist,
        "content": content,
        "compare": None,
        "keyword": parse[3].split(','),
        "fb_like": fb_like,
        "fb_share": fb_share,
        "fb_comment_count": fb_com,
        "category": parse[4].split(','),
        "comment": list(tree.values())
    }

In [5]:
import numpy as np
parse_content = [parser_page(urls[i]) for i in np.random.choice(len(urls), 10,replace=False)]
parse_content

[{'category': ['國際'],
  'comment': [],
  'compare': None,
  'content': '印尼雅加達14日發生恐怖攻擊事件，現在不只確定是ISIS在幕後操控，警方更鎖定一名激進份子「巴魯納伊姆」是最大主謀，因為他一年半前跑到ISIS在敘利亞的大本營「拉卡」受訓，一直籌備在印尼發動大型攻擊，希望藉此成為ISIS在東南亞的大頭目。外界更擔心的是，這次恐怖攻擊後，菲律賓和印尼的ISIS支持者會不會串聯起來，甚至把魔爪伸向馬來西亞和新加坡。▲警方進一步逮捕印尼恐攻案嫌犯，鎖定幕後黑手激進份子巴魯納伊姆。街角爆出巨響，目擊者忍不住驚呼。印尼恐攻案震驚國際，警方進一步逮捕數名嫌犯，同時鎖定幕後黑手，激進份子巴魯納伊姆。雅加達警察局長：「東南亞有個武裝分子叫巴魯納伊姆，他想當這地區的領袖，這就是他策畫攻擊的主因。」32歲的納伊姆戴著眼鏡長相斯文，擁有資訊工程學位，但2010年因非法持有武器入獄兩年半，一年前又跑去敘利亞加入ISIS伺機而動，就是想成為ISIS在東南亞地區的大頭目。▲專家表示雖然爆炸規模小，但ISIS魔爪已正式伸入印尼。恐怖主義專家：「這些爆炸裝置相對粗糙，可能是死亡人數較低的原因。」儘管規模不如幾個月前的巴黎恐攻案，但事件象徵ISIS魔爪已正式伸入全球最大穆斯林國家印尼。▲根據統計，目前已有數百名印尼人前往加入ISIS。印尼穆斯林人口高達2億1千萬人，過去就曾發生數起攻擊事件，根據統計，目前已有數百名印尼人前往加入ISIS，甚至有將近一成年輕人對ISIS表示支持，當局擔心，2002年奪走202條性命的峇里島爆炸案恐怕重演。▲當局擔心2002年的峇里島爆炸案重演。英國新聞記者：「問題是，這一串攻擊事件是敘利亞境內成員下的指示，或是受到ISIS啟發的本土聖戰士所發動的呢？這區域有幾個活躍的伊斯蘭聖戰組織，外界最大的擔憂就是他們合併與合作。」ISIS勢力深入東南亞，菲律賓阿布薩耶夫早就宣示效忠ISIS，整個東南亞陷入紅色警戒，馬來西亞、新加坡高度戒備。▲整個東南亞陷入紅色警戒，馬來西亞、新加坡高度戒備。。（圖／翻攝自NBC）國際事務編輯：「印尼官方已表示擔憂，擔心聖戰士從敘利亞和伊拉克返回家鄉，這會是個大問題，因為印尼國土不規則，有很多島嶼，邊境相對不容易管制。」官方統計，目前印尼國內有一千多名極端組織擁護者，這次的

In [ ]:
'http://www.setn.com/News.aspx?NewsID=117306&PageGroupID=6&ProjectID=987'
'http://www.setn.com/News.aspx?NewsID=117355&PageGroupID=4&ProjectID=1037'
'http://www.setn.com/News.aspx?NewsID=118782'

In [161]:
def handleDefines(x):
    return x

#eval(re.search('handleDefines\(\[\[\"PlatformVersions".*\["UserActionHistory"\]\]}\)',resp.text).group())

In [7]:
example = 'http://www.setn.com/News.aspx?NewsID=117306&PageGroupID=6&ProjectID=987'
tree = {}
fb_com_url = 'http://graph.facebook.com/comments?id=http://www.setn.com/News.aspx?NewsID='+ \
              re.findall('NewsID=([0-9]+)',example)[0]+ \
             '&limit=100&filter=stream&fields=parent.fields%28id%29,message,from,like_count,created_time,parent'


fb_data = requests.get(fb_com_url).json()
while True:
    if len(fb_data["data"])==0:
        break
    for comment in fb_data['data']:
        one_com = {}
        one_com["post_time"] = datetime.datetime(*map(lambda x: int(x),re.findall('[0-9]+',comment['created_time'][:-5])))
        one_com["actor"] = comment['from']['name']
        one_com["like"] = comment['like_count']
        one_com["content"] = comment['message']
        one_com["source_type"] = "facebook"
        one_com["dislike"] = None
        if 'parent' in comment:
            tree[comment['parent']['id']]["sub_comments"].append(one_com)
        else:
            one_com["sub_comments"] = []
            tree[comment['id']] = one_com
    
    fb_next_url = fb_com_url+'&after='+fb_data['paging']['cursors']['after']
    fb_data = requests.get(fb_next_url).json()
    

In [8]:
tree

{'1191274760900590_1191374734223926': {'actor': '王又將',
  'content': '柯P加油！',
  'dislike': None,
  'like': 4,
  'post_time': datetime.datetime(2016, 1, 8, 20, 19, 26),
  'source_type': 'facebook',
  'sub_comments': []},
 '1191274760900590_1191607537533979': {'actor': 'Sam Wen',
  'content': '阿伯超強',
  'dislike': None,
  'like': 1,
  'post_time': datetime.datetime(2016, 1, 9, 6, 25),
  'source_type': 'facebook',
  'sub_comments': []},
 '1191274760900590_1191816480846418': {'actor': '張夢雅',
  'content': '【僅.限.臺.灣】完美的夢中情人與你相約+L.INE:tea555sexy\n\n【看.主.頁約.妹】 性.福.S.kpye:mmlove.0510\n\n【見本人滿意再現金消.費喔】 \n\n★外送地區：【台北】【台中】【高雄】【新竹】【彰化】',
  'dislike': None,
  'like': 0,
  'post_time': datetime.datetime(2016, 1, 9, 14, 56, 25),
  'source_type': 'facebook',
  'sub_comments': []},
 '1191274760900590_1191838790844187': {'actor': 'Lan Ping Li',
  'content': '一路好走',
  'dislike': None,
  'like': 0,
  'post_time': datetime.datetime(2016, 1, 9, 15, 36, 4),
  'source_type': 'facebook',
  'sub_comments': []},
 '